# Part I: Single-Diode Model Calibration

In [ ]:
from pprint import pprint
import sys

import matplotlib
from matplotlib import pyplot
import numpy

import pvfit
from pvfit.common import E_hemispherical_tilted_W_per_m2_stc, T_degC_stc
from pvfit.measurement.iv.types import (
    FTData,
    IVCurve,
    IVPerformanceMatrix,
    SpecSheetParameters,
)
from pvfit.modeling.dc.common import Material
import pvfit.modeling.dc.single_diode.equation.simple.inference_iv_curve as sde_simple_inf
from pvfit.modeling.dc.single_diode.equation.simple.types import (
    ModelParameters,
    ModelParametersFittableFixedProvided,
    ModelParametersFittableProvided,
    ModelParametersUnfittable,
)
import pvfit.modeling.dc.single_diode.equation.simple.simulation as sde_simple_sim
import pvfit.modeling.dc.single_diode.model.photoconductive_shunt.auxiliary_equations as sdm_ps_ae
import pvfit.modeling.dc.single_diode.model.photoconductive_shunt.inference_matrix as sdm_ps_inf_matrix
import pvfit.modeling.dc.single_diode.model.simple.auxiliary_equations as sdm_simple_ae
import pvfit.modeling.dc.single_diode.model.simple.inference_matrix as sdm_simple_inf_matrix
import pvfit.modeling.dc.single_diode.model.simple.inference_spec_sheet as sdm_simple_inf_spec_sheet

print("Python version: %s.%s.%s" % sys.version_info[:3])
print("matplotlib version: " + matplotlib.__version__)
print("numpy version: " + numpy.__version__)
print("pvfit version: " + pvfit.__version__)

## Single-Diode Equation (SDE)

### Fitting the SDE to a single I-V curve at one temperature and irradiance.

#### I-V Curve Data

In [ ]:
# Special thanks to Cliff Hansen: https://github.com/cwhanse/ivcurves
V_V = numpy.array([
    0.0, 0.4014960341400983790, 0.8029920682801967580, 1.2044881024202951370, 1.6059841365603935159,
    2.0074801707004921170, 2.4089762048405902739, 2.8104722389806888749,3.2119682731207870319,
    3.6134643072608851888, 4.0149603414009842339, 4.4164563755410819468, 4.8179524096811805478,
    5.2194484438212791488, 5.6209444779613777499, 6.0224405121014754627, 6.4239365462415740637,
    6.8254325803816726648, 7.2269286145217703776, 7.6284246486618689786, 8.0299206828019684679,
    8.4314167169420652925, 8.8329127510821638936, 9.2344087852222624946, 9.6359048193623610956,
    10.0374008535024596966, 10.4388968876425582977, 10.8403929217826568987, 11.2418889559227554997,
    11.6433849900628523244, 12.0448810242029509254, 12.4463770583430495265, 12.8478730924831481275,
    13.2493691266232467285, 13.6508651607633453295, 14.0523611949034439306, 14.4538572290435407552,
    14.8553532631836393563, 15.2568492973237379573, 15.6583453314638365583, 16.0598413656039369357,
    16.4613373997440355367, 16.8628334338841305851, 17.2643294680242291861, 17.6658255021643277871,
    18.0673215363044263881, 18.4688175704445249892, 18.8703136045846235902, 19.2718096387247221912,
    19.6733056728648207923, 20.0748017070049193933, 20.4762977411450179943, 20.8777937752851165953,
    21.2792898094252116437, 21.6807858435653137974, 22.0822818777054088457, 22.4837779118455109995,
    22.8852739459856060478, 23.2867699801257046488, 23.6882660142658068025, 24.0897620484059018509,
    24.4912580825460040046, 24.8927541166860990529, 25.2942501508262012067, 25.6957461849662962550,
    26.0972422191063948560, 26.4987382532464934570, 26.9002342873865920581, 27.3017303215266906591,
    27.7032263556667892601, 28.1047223898068878611, 28.5062184239469864622, 28.9077144580870815105,
    29.3092104922271836642, 29.7107065263672787125, 30.1122025605073808663, 30.5136985946474759146,
    30.9151946287875745156, 31.3166906629276731167, 31.7181866970677717177, 32.1196827312078738714,
    32.5211787653479689197, 32.9226747994880710735, 33.3241708336281661218, 33.7256668677682611701,
    34.1271629019083633239, 34.5286589360484583722, 34.9301549701885605259, 35.3316510043286555742,
    35.7331470384687577280, 36.1346430726088527763, 36.5361391067489478246, 36.9376351408890499783,
    37.3391311750291521321, 37.7406272091692471804, 38.1421232433093493341, 38.5436192774494443825,
    38.9451153115895465362, 39.3466113457296415845, 39.7481073798697327059
])
I_A = numpy.array([
    0.9996667777132811507, 0.9983289034311699783, 0.9969910291187772389, 0.9956531547688463046,
    0.9943152803723819266, 0.9929774059182325807, 0.9916395313925747690, 0.9903016567782721887,
    0.9889637820540823077, 0.9876259071936717149, 0.9862880321643966441, 0.9849501569257905798,
    0.9836122814276898607, 0.9822744056079091290, 0.9809365293893617110, 0.9795986526764873803,
    0.9782607753508261598, 0.9769228972655307208, 0.9755850182385628767, 0.9742471380442615514,
    0.9729092564028880582, 0.9715713729676678234, 0.9702334873087252053, 0.9688955988931700266,
    0.9675577070604106655, 0.9662198109915521860, 0.9648819096714604746, 0.9635440018417359304,
    0.9622060859424174475, 0.9608681600397166456, 0.9595302217364341952, 0.9581922680609088472,
    0.9568542953293537323, 0.9555162989752035147, 0.9541782733375667735, 0.9528402113989826727,
    0.9515021044603342304, 0.9501639417378590979, 0.9488257098635876634, 0.9474873922660670411,
    0.9461489684026838098, 0.9448104128080203612, 0.9434716939141639669, 0.9421327725883148426,
    0.9407936003199529766, 0.9394541169735817011, 0.9381142480029479591, 0.9367739009976898679,
    0.9354329614024401440, 0.9340912872100813928, 0.9327487023833307411, 0.9314049886999290484,
    0.9300598756436964099, 0.9287130278732068644, 0.9273640296876487443, 0.9260123657703739353,
    0.9246573973182737882, 0.9232983324514715262, 0.9219341895330183112, 0.9205637517000882442,
    0.9191855105014244964, 0.9177975960317263926, 0.9163976903290566023, 0.9149829200274038956,
    0.9135497232976972185, 0.9120936849227954113, 0.9106093318808918813, 0.9090898799901701836,
    0.9075269199125221009, 0.9059100280234794189, 0.9042262842034205494, 0.9024596743369887042,
    0.9005903500339722534, 0.8985937115738941652, 0.8964392720458767008, 0.8940892507624991952,
    0.8914968318583503804, 0.8886040090464338848, 0.8853389192126983609, 0.8816125451955969166,
    0.8773146409370268239, 0.8723086993230496633, 0.8664257435102087346, 0.8594566753989285601,
    0.8511428593089253036, 0.8411645542528536539, 0.8291267344958724861, 0.8145417563918237912,
    0.7968082426514525585, 0.7751854689456849277, 0.7487624622220575138, 0.7164209720215719456,
    0.6767914814885996888, 0.6282015224566096512, 0.5686158040313862294, 0.4955681305251634626,
    0.4060858652321705779, 0.2966088961935733891, 0.1629067767190341108, 0.0
])

iv_curve = IVCurve(V_V=V_V, I_A=I_A)

fig, ax = pyplot.subplots()
ax.plot(iv_curve.V_V, iv_curve.I_A, '.')
ax.set_xlabel("V [V]")
ax.set_ylabel("I [A]")
ax.set_title("I-V Curve Data to Fit")
pass

#### Fit simple SDE (no reverse breakdown or bypass diodes)

In [ ]:
model_parameters_unfittable = ModelParametersUnfittable(N_s=72, T_degC=25)

sde_simple_fit = sde_simple_inf.fit(iv_curve=iv_curve, model_parameters_unfittable=model_parameters_unfittable)
pprint(sde_simple_fit)

fig, ax = pyplot.subplots()
ax.plot(iv_curve.V_V, iv_curve.I_A, '.', label="Measured I-V Data")
ax.plot(
    iv_curve.V_V,
    sde_simple_sim.I_at_V(V_V=iv_curve.V_V, model_parameters=sde_simple_fit["model_parameters_ic"])["I_A"],
    label="Initial Condition",
)
ax.plot(
    iv_curve.V_V,
    sde_simple_sim.I_at_V(V_V=iv_curve.V_V, model_parameters=sde_simple_fit["model_parameters"])["I_A"],
    label="Fit",
)
ax.legend()
ax.set_xlabel("V [V]")
ax.set_ylabel("I [A]")
ax.set_title("I-V Curve Data Fit")
pass

#### Inspecting the solver's underlying solution

In [ ]:
# CAUTION: The solution found by scipy's orthogonal distance regression (ODR) is for a transformed problem.
sde_simple_fit["odr_output"].beta, sde_simple_fit["odr_output"].stopreason

#### Simulating performance with the fit result

In [ ]:
sdm_simple_fit_matrix_iv_curve_parameters = sde_simple_sim.iv_curve_parameters(model_parameters=sde_simple_fit["model_parameters"])
pprint(sdm_simple_fit_matrix_iv_curve_parameters)

fig, ax = pyplot.subplots()
ax.plot(iv_curve.V_V, sde_simple_sim.I_at_V(V_V=iv_curve.V_V, model_parameters=sde_simple_fit["model_parameters"])["I_A"], label="Fit")
ax.plot(0, sdm_simple_fit_matrix_iv_curve_parameters["I_sc_A"], 'x', label="(0, I_sc)",)
ax.plot(sdm_simple_fit_matrix_iv_curve_parameters["V_x_V"], sdm_simple_fit_matrix_iv_curve_parameters["I_x_A"], 'x', label="(V_x, I_x)")
ax.plot(sdm_simple_fit_matrix_iv_curve_parameters["V_mp_V"], sdm_simple_fit_matrix_iv_curve_parameters["I_mp_A"], 'x', label="P_mp @ (V_mp, I_mp)")
ax.plot(sdm_simple_fit_matrix_iv_curve_parameters["V_xx_V"], sdm_simple_fit_matrix_iv_curve_parameters["I_xx_A"], 'x', label="(V_xx, I_xx)")
ax.plot(sdm_simple_fit_matrix_iv_curve_parameters["V_oc_V"], 0, 'x', label="(V_oc, 0)")
ax.legend()
ax.set_xlabel("V [V]")
ax.set_ylabel("I [A]")
ax.set_title("Fit's Performance Points")
pass

#### Vectorization over model parameters too

In [ ]:
# Compare fill factor, maximum power, etc. to ideal and degraded device.
model_parameters_vectorized = ModelParameters(**sde_simple_fit["model_parameters"])
model_parameters_vectorized["R_s_Ohm"] = numpy.array(
    [
        0.0,
        sde_simple_fit["model_parameters"]["R_s_Ohm"],
        10*sde_simple_fit["model_parameters"]["R_s_Ohm"]
    ]
)
print("model_parameters_vectorized:")
pprint(model_parameters_vectorized)
# Note that fill factor function returns the addtional values used to calculate FF.
print("FF computation:")
pprint(sde_simple_sim.FF(model_parameters=model_parameters_vectorized))

#### Fit with initial conditions provided for select parameters

PVfit tries hard to guess a good starting value, but you can partially/fully override its choices.

In [ ]:
sde_simple_fit_with_ic = sde_simple_inf.fit(
    iv_curve=iv_curve,
    model_parameters_unfittable=model_parameters_unfittable,
    model_parameters_fittable_ic_provided=ModelParametersFittableProvided(
        n=1.0,
        I_rs_A=5.0e-10,
        R_s_Ohm=0.2,
    )
)
pprint(sde_simple_fit_with_ic)

fig, ax = pyplot.subplots()
ax.plot(iv_curve.V_V, iv_curve.I_A, '.', label="Measured I-V Data")
ax.plot(iv_curve.V_V, sde_simple_sim.I_at_V(V_V=iv_curve.V_V, model_parameters=sde_simple_fit_with_ic["model_parameters_ic"])["I_A"], label="Initial Condition")
ax.plot(iv_curve.V_V, sde_simple_sim.I_at_V(V_V=iv_curve.V_V, model_parameters=sde_simple_fit_with_ic["model_parameters"])["I_A"], label="Fit")
ax.legend()
ax.set_xlabel("V [V]")
ax.set_ylabel("I [A]")
ax.set_title("I-V Curve Data Fit with Some ICs Provided")
pass

#### Fit with fixed ideality factor (non-ideal diode)

Here, we provide two ICs, but only fix one of them. (The underlying ODR solver makes this easy to implement.)

In [ ]:
sde_simple_fit_with_ic_fixed = sde_simple_inf.fit(
    iv_curve=iv_curve,
    model_parameters_unfittable=model_parameters_unfittable,
    model_parameters_fittable_ic_provided=ModelParametersFittableProvided(
        n=1.5,
        I_rs_A=5.0e-8,
    ),
    model_parameters_fittable_fixed_provided=ModelParametersFittableFixedProvided(
        n=True  
    )
)
pprint(sde_simple_fit_with_ic_fixed)

fig, ax = pyplot.subplots()
ax.plot(iv_curve.V_V, iv_curve.I_A, '.', label="Measured I-V Data")
ax.plot(iv_curve.V_V, sde_simple_sim.I_at_V(V_V=iv_curve.V_V, model_parameters=sde_simple_fit_with_ic_fixed["model_parameters_ic"])["I_A"], label="Initial Condition")
ax.plot(iv_curve.V_V, sde_simple_sim.I_at_V(V_V=iv_curve.V_V, model_parameters=sde_simple_fit_with_ic_fixed["model_parameters"])["I_A"], label="Fit")
ax.legend()
ax.set_xlabel("V [V]")
ax.set_ylabel("I [A]")
ax.set_title("I-V Curve Data Fit with n=1 Fixed")
pass

## Single-Diode Model (SDM)

### Fitting the SDM (multiple temperature and irradiance).

#### IEC 61853-1 (T, E) matrix data


In [ ]:
# Special thanks to PVPMC: https://pvpmc.sandia.gov/pv-research/pv-lifetime-project/pv-lifetime-modules/
# LG320N1K-A5 320W N-PERT Si, CFV Report # 19074-PR-E-005
I_sc_A = numpy.array([
    1.02432156592117, 2.04220329161805, 4.08876231489616, 6.16742104333028, 8.23620585618444, 10.3278357876755,
    1.02757453945704, 2.04102689488776, 4.0971414004165, 6.18719186830119, 8.26393082981699, 10.3551722873482, 11.395279503447,
    1.03913626907321, 2.06392255863692, 4.13658036822046, 6.23104469564041, 8.32545975924577, 10.4337969237837, 11.4824044494735,
    1.04491230559491, 2.0826858185294, 4.17166165651346, 6.27316031746301, 8.38355926709284, 10.4998465959987, 11.5723836702704,
])
I_mp_A = numpy.array([
    0.97751989541629, 1.95726960371108, 3.92418023622383, 5.88128100651897, 7.83318220601609, 9.79347790957467,
    0.98916629640619, 1.94985466389299, 3.91453936347665, 5.86450863231455, 7.82315841412953, 9.78856875676654, 10.7347437361592,
    0.979284585841039, 1.9562110868987, 3.89343224765022, 5.85540132593549, 7.80002485702661, 9.7568852648073, 10.7216370906218,
    0.974720372318489, 1.952125140497, 3.88896895682736, 5.83030275932059, 7.76392690540913, 9.68589365355965, 10.6291077113922,
])
V_mp_V = numpy.array([
    32.483559723954, 33.258802022921, 33.8340393593184, 34.0645574854619, 34.0867190147778, 33.9728268087745,
    30.9778660821221, 31.9811420769189, 32.594898159062, 32.8194215533058, 32.8550150165226, 32.7239339231629, 32.7049519389132,
    27.8308170148312, 28.7322982525907, 29.5538016200052, 29.6987353242727, 29.6990451824235, 29.5689893366802, 29.4925560748204,
    24.4491348564815, 25.448012412096, 26.219831977235, 26.427468974117, 26.4770877509191, 26.4207371130951, 26.4033443018523,
])
V_oc_V = numpy.array([
    37.5801692934928, 38.6911107010062, 39.8079483178206, 40.4518064164751, 40.9107513118948, 41.2545791691551,
    36.3299943128803, 37.469502764561, 38.62911737847, 39.2968966959977, 39.7727741849426, 40.1323305920718, 40.2987316268181,
    33.1224743848133, 34.3345551259948, 35.5886134033995, 36.3181133114656, 36.8413940279752, 37.2287178272536, 37.4032160396215,
    29.7485796556728, 31.0865085511878, 32.4495485404176, 33.2484353792992, 33.8118471371682, 34.2675187516404, 34.4526444363561,
])
E_W_per_m2 = numpy.array([
    100, 200, 400, 600, 800, 1000,
    100, 200, 400, 600, 800, 1000, 1100,
    100, 200, 400, 600, 800, 1000, 1100,
    100, 200, 400, 600, 800, 1000, 1100,
])  # This is called G in IEC 61853-1, which we assume is direct illumination.
T_degC = numpy.array([
    15, 15, 15, 15, 15, 15,
    25, 25, 25, 25, 25, 25, 25,
    50, 50, 50, 50, 50, 50, 50,
    75, 75, 75, 75, 75, 75, 75,
])  # This is called T in IEC 61853-1, which we assume is cell temperature (not back of module).

iv_performance_matrix = IVPerformanceMatrix(
    material=Material.monoSi,
    N_s=60,
    I_sc_A=I_sc_A,
    I_mp_A=I_mp_A,
    V_mp_V=V_mp_V,
    V_oc_V=V_oc_V,
    E_W_per_m2=E_W_per_m2,
    T_degC=T_degC,
    E_W_per_m2_0=E_hemispherical_tilted_W_per_m2_stc,
    T_degC_0=T_degC_stc,
)

#### Fit Simple SDM (uses simple SDE with constant ideality factor, series resistance, and parallel conductance)

In [ ]:
sdm_simple_fit_matrix = sdm_simple_inf_matrix.fit(iv_performance_matrix=iv_performance_matrix)

pprint(sdm_simple_fit_matrix["model_parameters"])
pprint(sdm_simple_inf_matrix.compute_fit_quality(
    iv_performance_matrix=iv_performance_matrix,
    model_parameters=sdm_simple_fit_matrix["model_parameters"],
)[0])

##### Plot fit

In [ ]:
sdm_simple_fit_matrix_iv_curve_parameters = sde_simple_sim.iv_curve_parameters(
    model_parameters=sdm_simple_ae.compute_sde_model_parameters(
        ft_data=FTData(F=iv_performance_matrix.F, T_degC=iv_performance_matrix.T_degC),
        model_parameters=sdm_simple_fit_matrix["model_parameters"],
    )
)

# Plot the data fits.
fig, ax = pyplot.subplots(figsize=(8, 6))
for idx, (F, T_degC) in enumerate(
    zip(iv_performance_matrix.F, iv_performance_matrix.T_degC)
):
    # Plot Isc, Pmp, and Voc with same colors as fit lines.
    color = next(ax._get_lines.prop_cycler)["color"]
    ax.plot(
        iv_performance_matrix.ivft_data.V_V[3 * idx : 3 * idx + 3],
        iv_performance_matrix.ivft_data.I_A[3 * idx : 3 * idx + 3],
        "o",
        color=color,
        mfc='none',
    )
    V_V_plot = numpy.linspace(0, iv_performance_matrix.ivft_data.V_V[3 * idx + 2], 101)
    ax.plot(
        V_V_plot,
        sde_simple_sim.I_at_V(
            V_V=V_V_plot,
            model_parameters=sdm_simple_ae.compute_sde_model_parameters(
                ft_data=FTData(F=F, T_degC=T_degC),
                model_parameters=sdm_simple_fit_matrix["model_parameters"],
            ),
        )["I_A"],
        label=f"F={F:.2f}, T={T_degC:.0f} °C",
        color=color,
    )
    ax.plot(0, sdm_simple_fit_matrix_iv_curve_parameters["I_sc_A"][idx], 'x', color=color)
    ax.plot(
        sdm_simple_fit_matrix_iv_curve_parameters["V_mp_V"][idx],
        sdm_simple_fit_matrix_iv_curve_parameters["I_mp_A"][idx],
        'x', color=color
    )
    ax.plot(sdm_simple_fit_matrix_iv_curve_parameters["V_oc_V"][idx], 0, 'x', color=color)

ax.set_title("6-Parameter Simple SDM Fit to Performance Matrix", fontdict={"fontsize": 14})
ax.set_xlabel("V (V)")
ax.set_ylabel("I (A)")
fig.legend(bbox_to_anchor=(1.15, 1))

pyplot.show()

#### Fit Photoconductive-Shunt SDM

Uses simple SDE with constant ideality factor and series resistance and shunt conductance proportional to effective irradiance ratio.

In [ ]:
sdm_ps_fit_matrix = sdm_ps_inf_matrix.fit(iv_performance_matrix=iv_performance_matrix)

pprint(sdm_ps_fit_matrix["model_parameters"])
pprint(sdm_ps_inf_matrix.compute_fit_quality(
    iv_performance_matrix=iv_performance_matrix,
    model_parameters=sdm_ps_fit_matrix["model_parameters"],
)[0])

##### Plot fit

In [ ]:
sdm_ps_fit_matrix_iv_curve_parameters = sde_simple_sim.iv_curve_parameters(
    model_parameters=sdm_ps_ae.compute_sde_model_parameters(
        ft_data=FTData(F=iv_performance_matrix.F, T_degC=iv_performance_matrix.T_degC),
        model_parameters=sdm_ps_fit_matrix["model_parameters"],
    )
)

fig, ax = pyplot.subplots(figsize=(8, 6))
# Plot the data fits.
for idx, (F, T_degC) in enumerate(
    zip(iv_performance_matrix.F, iv_performance_matrix.T_degC)
):
    # Plot Isc, Pmp, and Voc with same colors as fit lines.
    color = next(ax._get_lines.prop_cycler)["color"]
    ax.plot(
        iv_performance_matrix.ivft_data.V_V[3 * idx : 3 * idx + 3],
        iv_performance_matrix.ivft_data.I_A[3 * idx : 3 * idx + 3],
        "o",
        color=color,
        mfc='none',

    )
    V_V_plot = numpy.linspace(0, iv_performance_matrix.ivft_data.V_V[3 * idx + 2], 101)
    ax.plot(
        V_V_plot,
        sde_simple_sim.I_at_V(
            V_V=V_V_plot,
            model_parameters=sdm_ps_ae.compute_sde_model_parameters(
                ft_data=FTData(F=F, T_degC=T_degC),
                model_parameters=sdm_ps_fit_matrix["model_parameters"],
            ),
        )["I_A"],
        label=f"F={F:.2f}, T={T_degC:.0f} °C",
        color=color,
    )
    ax.plot(0, sdm_ps_fit_matrix_iv_curve_parameters["I_sc_A"][idx], 'x', color=color)
    ax.plot(
        sdm_ps_fit_matrix_iv_curve_parameters["V_mp_V"][idx],
        sdm_ps_fit_matrix_iv_curve_parameters["I_mp_A"][idx],
        'x', color=color
    )
    ax.plot(sdm_ps_fit_matrix_iv_curve_parameters["V_oc_V"][idx], 0, 'x', color=color)

ax.set_title("6-Parameter Photoconductive-Shunt SDM Fit to Performance Matrix", fontdict={"fontsize": 14})
ax.set_xlabel("V (V)")
ax.set_ylabel("I (A)")
fig.legend(bbox_to_anchor=(1.15, 1))

pyplot.show()

#### Specification Datasheet Data at Standard Test Conditions (STC) with Temperature Coefficients

#### Spec Sheet Data (for apples-to-apples, taken from IEC 61853-1 report)

In [ ]:
# 320 W nominal mono-Si module.
# These values are actually taken from the IEC 61853-1 measurments for a better fit comparison.
I_sc_A_0 = 10.3551722873482  # 10.19 A nominal
I_mp_A_0 = 9.78856875676654  # 9.62 A nominal
V_mp_V_0 = 32.7239339231629  # 33.3 V nominal
V_oc_V_0 = 40.1323305920718  # 40.8 V nominal
# Separate temperature coefficient measurements, with absolute units.
dI_sc_dT_A_per_degC_0 = 0.00306  # 0.03 %/degC nominal
dP_mp_dT_W_per_degC_0 = -1.2817  # -0.37 %/degC nominal
dV_oc_dT_V_per_degC_0 = -0.1168  # -0.27 %/degC nominal

spec_sheet_parameters = SpecSheetParameters(
    material=Material.monoSi,
    N_s=60,
    I_sc_A_0=I_sc_A_0,
    I_mp_A_0=I_mp_A_0,
    V_mp_V_0=V_mp_V_0,
    V_oc_V_0=V_oc_V_0,
    dI_sc_dT_A_per_degC_0=dI_sc_dT_A_per_degC_0,
    dP_mp_dT_W_per_degC_0=dP_mp_dT_W_per_degC_0,
    dV_oc_dT_V_per_degC_0=dV_oc_dT_V_per_degC_0,
    E_W_per_m2_0=E_hemispherical_tilted_W_per_m2_stc,
    T_degC_0=T_degC_stc,
)

#### Fit Simple SDM to Spec Sheet Data

In [ ]:
sdm_simple_fit_spec_sheet = sdm_simple_inf_spec_sheet.fit(spec_sheet_parameters=spec_sheet_parameters)

pprint(sdm_simple_fit_spec_sheet["model_parameters"])
pprint(sdm_simple_inf_spec_sheet.compute_fit_quality(
    iv_performance_matrix=iv_performance_matrix,
    model_parameters=sdm_simple_fit_spec_sheet["model_parameters"],
)[0])

##### Plot fit

In [ ]:
sdm_simple_fit_spec_sheet_iv_curve_parameters = sde_simple_sim.iv_curve_parameters(
    model_parameters=sdm_simple_ae.compute_sde_model_parameters(
        ft_data=FTData(F=iv_performance_matrix.F, T_degC=iv_performance_matrix.T_degC),
        model_parameters=sdm_simple_fit_spec_sheet["model_parameters"],
    )
)

fig, ax = pyplot.subplots(figsize=(8, 6))
# Plot the data fits.
for idx, (F, T_degC) in enumerate(
    zip(iv_performance_matrix.F, iv_performance_matrix.T_degC)
):
    # Plot Isc, Pmp, and Voc with same colors as fit lines.
    color = next(ax._get_lines.prop_cycler)["color"]
    ax.plot(
        iv_performance_matrix.ivft_data.V_V[3 * idx : 3 * idx + 3],
        iv_performance_matrix.ivft_data.I_A[3 * idx : 3 * idx + 3],
        "o",
        color=color,
        mfc='none',
    )
    V_V_plot = numpy.linspace(0, iv_performance_matrix.ivft_data.V_V[3 * idx + 2], 101)
    ax.plot(
        V_V_plot,
        sde_simple_sim.I_at_V(
            V_V=V_V_plot,
            model_parameters=sdm_simple_ae.compute_sde_model_parameters(
                ft_data=FTData(F=F, T_degC=T_degC),
                model_parameters=sdm_simple_fit_spec_sheet["model_parameters"],
            ),
        )["I_A"],
        label=f"F={F:.2f}, T={T_degC:.0f} °C", color=color,
    )
    ax.plot(0, sdm_simple_fit_spec_sheet_iv_curve_parameters["I_sc_A"][idx], 'x', color=color)
    ax.plot(
        sdm_simple_fit_spec_sheet_iv_curve_parameters["V_mp_V"][idx],
        sdm_simple_fit_spec_sheet_iv_curve_parameters["I_mp_A"][idx],
        'x', color=color
    )
    ax.plot(sdm_simple_fit_spec_sheet_iv_curve_parameters["V_oc_V"][idx], 0, 'x', color=color)

ax.set_title("6-Parameter Simple SDM Fit to Spec Sheet", fontdict={"fontsize": 14})
ax.set_xlabel("V (V)")
ax.set_ylabel("I (A)")
fig.legend(bbox_to_anchor=(1.15, 1))

pyplot.show()